# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=15, debug_line = False):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if debug_line:
        # draw original input lines without any modification
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    else:
        # draw lanes from input lines with extrapolating, averaging, and other tricks
        fit_lefts = []; fit_rights = []
        #weight_lefts = []; weight_rights = []
        y2_min = image.shape[0]  # image shape[0] in mping.imread corresponds　y axis in cv2
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x2 != x1: # ignore holizontal line
                    slope = -(y2-y1)/(x2-x1) # to align with our intuition set negative for slope formula
                    #print("slope: ", slope)
                    if (0.8 > slope > 0.4) and (max(x1, x2) < image.shape[1]/2.):  # increasing slope and left hand side
                        fits = np.polyfit((x1, x2), (y1, y2), 1)
                        if not any(np.isnan(fits)):
                            fit_lefts.append(fits)
                            #weight_lefts.append(np.sqrt(np.square(x1-x2) + np.square(y1-y2)))
                    elif (-0.8 < slope < -0.4) and (min(x1, x2) > image.shape[1]/2.): # decreasing slope and right hand side
                        fits = np.polyfit((x1, x2), (y1, y2), 1)
                        if not any(np.isnan(fits)):
                            fit_rights.append(fits)
                            #weight_rights.append(np.sqrt(np.square(x1-x2) + np.square(y1-y2)))
                    if y2_min > y2: 
                        y2_min = y2

        # remove outliers based on slope
        def reject_outliers(data, m = 1.1):
            if len(data) == 1:
                return data
            slopes = data[:, 0]
            d = np.abs(slopes - np.median(slopes))
            mdev = np.median(d)
            s = d/mdev if mdev else 0.
            if type(s).__module__ == np.__name__: # if s is numpy array
                return data[s<m]
            else:
                return data

        # draw left lane
        if len(fit_lefts) > 0:
            #fit_left = np.average(np.array(fit_lefts), axis=0, weights=weight_lefts)
            fit_left = np.average(reject_outliers(np.array(fit_lefts)), axis=0)
            if not np.isnan((y1 - fit_left[1]) / fit_left[0]):
                y1 = img.shape[0]; x1 = int((y1 - fit_left[1]) / fit_left[0])
                y2 = y2_min; x2 = int((y2 - fit_left[1]) / fit_left[0])
                if (abs(fit_left[0]) > 1e-5): # exclude when slope is very small 
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)

        # draw right lane
        if len(fit_rights) > 0:
            #fit_right = np.average(np.array(fit_rights), axis=0, weights=weight_rights)
            fit_right = np.average(reject_outliers(np.array(fit_rights)), axis=0)
            if not np.isnan((y1 - fit_right[1]) / fit_right[0]):
                y1 = img.shape[0]; x1 = int((y1 - fit_right[1]) / fit_right[0])
                y2 = y2_min; x2 = int((y2 - fit_right[1]) / fit_right[0])
                if (abs(fit_right[0]) > 1e-5): # exclude when slope is very small 
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, debug_line=False):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, debug_line=debug_line)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

def find_lane_lines(initial_img, kernel_size=5, low_threshold=50, high_threshold=150,
                   vertices=np.array([], dtype=np.int32).any(), rho=2, theta=np.pi/180, threshold=15,
                   min_line_len=40, max_line_gap=20, α=0.8, β=1., λ=0.,
                    debug_check=False, show_initial_img=False, debug_line=False):
    # Read in and grayscale the image
    gray = grayscale(initial_img)
    # Apply Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size)
    # Applies the Canny transform
    edges = canny(blur_gray, low_threshold, high_threshold)
    # Create a masked edges image
    if not vertices.any():
        imshape = initial_img.shape
        vertices = np.array([[(0,imshape[0]),(0, 0), (imshape[1], 0), 
                              (imshape[1],imshape[0])]], dtype=np.int32)
        vertices = np.array([[(0, imshape[0]),(imshape[1]/3., imshape[0]*2/3.), 
                                              (imshape[1]*2/3., imshape[0]*2/3.), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
                              
    # Create an image with hough lines drawn
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, 
                             max_line_gap, debug_line=debug_line)
    
    lane_image = weighted_img(line_image, initial_img, α=α, β=β, λ=λ)

    if debug_check:
        channel_count = initial_img.shape[2]
        color_edges = np.dstack((edges for i in range(channel_count)))
        lines_edges = weighted_img(line_image, color_edges, α=α, β=β, λ=λ)
        fig, axs = plt.subplots(1, 2, figsize=(13, 8))
        axs[0].imshow(lines_edges)
        if show_initial_img:
            axs[1].imshow(initial_img)
        else:
            axs[1].imshow(lane_image)
        plt.show()

    return lane_image

In [ ]:
dirname = 'test_images'

kernel_size=5 # kernel size for Gaussian smoothing
low_threshold=50 # low threshold for Canny and apply
high_threshold=150 # high threshold for Canny and apply
vertices= np.array([], dtype=np.int32).any() # defining a four sided polygon to mask
#vertices = np.array([[(0, 540),(440, 330), (500, 320), (960,540)]], dtype=np.int32)
#vertices = np.array([[(100, 540),(440, 330), (500, 330), (960,540)]], dtype=np.int32)
rho = 1 #1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 2 #10     # minimum number of votes (intersections in Hough grid cell)
min_line_len = 20 #50 #minimum number of pixels making up a line
max_line_gap = 6 #6    # maximum gap in pixels between connectable line segments
α=0.8 # make the lines semi-transparent
β=1. # make the inital image semi-transparent
λ=0. # make the lines and initial image semi-transparent

for fname in os.listdir(dirname):
    print(fname, end=' ')
    image = mpimg.imread(os.path.join(dirname, fname))
    print(image.shape)
    image_with_lanes = find_lane_lines(image, kernel_size=kernel_size, low_threshold=low_threshold, high_threshold=high_threshold, 
                                                                    vertices=vertices, rho=rho, theta=theta, threshold=threshold, 
                                                                    min_line_len=min_line_len, max_line_gap=max_line_gap, α=α, β=β, λ=λ, 
                                                                    debug_check=True, show_initial_img=False, debug_line=False)
    mpimg.imsave(os.path.join(dirname, fname.split('.')[0] + "_with_lanes.jpg"), image_with_lanes) # save result

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    result = find_lane_lines(image, kernel_size=kernel_size, low_threshold=low_threshold, high_threshold=high_threshold, 
                        vertices=vertices, rho=rho, theta=theta, threshold=threshold, 
                        min_line_len=min_line_len, max_line_gap=max_line_gap, α=α, β=β, λ=λ)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))